# Libraries

In [69]:
import cobra as cb
import logging
logging.basicConfig(filename="log.txt"  , level=logging.INFO)
import os
import pandas as pd
import numpy as np
import itertools
from itertools import combinations_with_replacement
import scipy.stats 

# Set up

In [52]:
modelNames = ["ENGRO 1", "ENGRO 2"]

In [53]:
##############################################
# Create a folder if it doesn't already exist
# Parameters
# - path --> new folder path
##############################################
def createFolder(path):
    if not os.path.exists(path):
            os.mkdir(path)

# Loading models

In [54]:
##############################################
# Load the models files (.xml)
# Parameters
# - modelNames --> list of models names that must
# match the file names
# - modelFolder --> the folder containing the 
# models files
##############################################
def loadModels(modelNames, modelFolder):
    models = {}
    for modelName in modelNames:
        files = os.listdir(modelFolder)
        found = False
        for file in files:
            filename, extension = os.path.splitext(file)
            if(filename == modelName):
                found = True
                break
        if(found):
            if(extension == ".xml"):
                models[modelName] = cb.io.read_sbml_model(modelFolder + filename + extension)
            else:
                raise ImportError('Model file extension not supported')
        else:
            raise FileNotFoundError('File not found')
    return models

In [55]:
modelsDict = loadModels(modelNames, "../../models/")
modelReactionsDict = {}
for modelName in modelNames:
    listReactions = []
    for reaction in modelsDict[modelName].reactions:
        listReactions.append(reaction.id)
    modelReactionsDict[modelName] = listReactions

# KLD analysis

In [ ]:
def kldAnalysis(firstNSample, lastNSample,stepSize ,
                numberOfTestPerNSample, filePath,  resultPath):
    df = pd.read_csv(filePath, index_col = 1)
    df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],
            axis = 1, inplace = True)
    nReaction = len(modelReactions)
    blockedR = cb.flux_analysis.find_blocked_reactions(getModel())
    nBlockedR = len(blockedR)
    columns = ["Samples"]
    for col in df.columns:
        columns.append(col)
    columns.append("Total")
    dfResult = pd.DataFrame(columns =  columns)
    
    for column in df.columns:
        summ = 0
        i = 0
        res = []
        for cell in df[column]:
            summ = summ + cell
            i = i + 1
            if(i == numberOfTestPerNSample):
                res.append(summ/numberOfTestPerNSample)
                i = 0
                summ = 0
        dfResult[column] = res
        
    sampleList = []
    for i in range (firstNSample, lastNSample, stepSize):
        sampleList.append(i)
    dfResult["Samples"] = sampleList
    dfResult = dfResult.set_index("Samples")
    res = []
    for index, row in dfResult.iterrows():
        summ = 0
        for i in range(0, nReaction):
            if(modelReactions[i] not in blockedR):
                summ = summ + row[i]
        res.append(summ/(nReaction - nBlockedR))
    dfResult["Total"] = res
    dfResult.to_csv(resultPath)
    pass